In [1]:
import pandas as pd
import numpy as np
from processer import fragmentSmilesList, getSelfiesToks
from multiprocesspandas import applyparallel


## dataset Moses Tokenization

In [2]:
pathData = "experiments/data/"

In [3]:
path="~/.local/lib/python3.11/site-packages/molsets-1.0-py3.11.egg/moses/dataset/data/"

In [4]:
mosesData = np.loadtxt(path+"dataset_v1.txt", dtype=str, usecols=[0], comments=None, delimiter="," ,skiprows=1)

In [5]:
mosesData.size

1936962

In [8]:
mosesDataToks = fragmentSmilesList(mosesData)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1936962/1936962 [00:30<00:00, 62653.24it/s]


In [10]:
mosesDataToks.dropna(inplace=True)
mosesDataToks.rename(columns={'tokens':'fragsmiles'},inplace=True)
mosesDataToks.drop(columns='nToks', inplace=True)

In [11]:
mosesDataToks['selfies']=mosesDataToks['smiles'].apply_parallel(getSelfiesToks, num_processes=30)

100%|████████████████████████████████████████████████████████████████████████████████████████| 1936693/1936693 [00:03<00:00, 489814.56it/s]


In [14]:
mosesDataToks

,smiles,fragsmiles,selfies
0,CCCS(=O)c1ccc2[nH]c(=NC(=O)OC)[nH]c2c1,"(C, C, C, O=[SH2], <6>, N=c1[nH]c2ccccc2[nH]1,...","([C], [C], [C], [S], [=Branch1], [C], [=O], [C..."
1,CC(C)(C)C(=O)C(Oc1ccc(Cl)cc1)n1ccnc1,"(C, C, (, C, ), (, C, ), C=O, C, (, <2>, c1c[n...","([C], [C], [Branch1], [C], [C], [Branch1], [C]..."
2,CC1C2CCC(C2)C1CN(CCO)C(=O)c1ccc(Cl)cc1,"(C, <4>, C1CC2CCC1C2, <3>, C, N, (, C, C, O, )...","([C], [C], [C], [C], [C], [C], [Branch1], [Rin..."
3,Cc1c(Cl)cccc1Nc1ncccc1C(=O)OCC(O)CO,"(C, <5>, c1ccccc1, <4>, <0>, (, Cl, ), N, <2>,...","([C], [C], [=C], [Branch1], [C], [Cl], [C], [=..."
4,Cn1cnc2c1c(=O)n(CC(O)CO)c(=O)n2C,"(C, <6>, O=c1[nH]c(=O)c2[nH]cnc2[nH]1, <2>, <1...","([C], [N], [C], [=N], [C], [=C], [Ring1], [Bra..."
...,...,...,...
1936957,N#Cc1c(Br)cnc(N)c1Br,"(C#N, <0>, c1ccncc1, <1>, <4>, (, N, ), <5>, (...","([N], [#C], [C], [=C], [Branch1], [C], [Br], [..."
1936958,COC(=O)c1cc(CNC(=O)OC(C)(C)C)ccc1C,"(C, C, (, C, ), (, C, ), O, C=O, N, C, <0>, c1...","([C], [O], [C], [=Branch1], [C], [=O], [C], [=..."
1936959,NC(=O)c1ccc2ccccc2c1Br,"(N, C=O, <0>, c1ccc2ccccc2c1, <9>, Br)","([N], [C], [=Branch1], [C], [=O], [C], [=C], [..."
1936960,CC(=O)Nc1cccc(-c2nc3cc(C)ccc3[nH]c2=O)c1,"(C, C=O, N, <0>, c1ccccc1, <4>, <2>, O=c1cnc2c...","([C], [C], [=Branch1], [C], [=O], [N], [C], [=..."


In [12]:
## 269 are molecules composed by only 1 fragment, so they are not considered as fragSmiles
len(mosesData)-len(mosesDataToks)

269

In [16]:
from sklearn.model_selection import KFold
def saveSplittedDataset(dataset, suffix=''):

    indexes = dataset.index.unique()

    kf=KFold(n_splits=5, shuffle=True, random_state=0)

    kf = kf.split( indexes )

    for fold,(trainIdx,testIdx) in enumerate(kf):
        train = dataset.loc[indexes[trainIdx]]
        test = dataset.loc[indexes[testIdx]]

        train.to_csv(pathData+f"moses_train{suffix}_{fold}.tar.xz", compression="xz", index=False)
        test.to_csv(pathData+f"moses_test{suffix}_{fold}.tar.xz", compression="xz", index=False)

In [17]:
saveSplittedDataset(mosesDataToks)